In [3]:
import os
os.environ["LSD_CACHEDIR"] = os.environ["HOME"] + "/tempdir"
os.environ["LSD_DB"] = os.environ["HOME"] + "/lsddb"
os.environ["LSD_TEMPDIR"] = os.environ["HOME"] + "/tempdir"
os.environ["NWORKERS"] = "8"

pathToRaveData = '/astro/users/scw7/Dropbox/Classwork/ast597b/final/data/RAVE_SAMPLE.csv'

In [5]:
from astropy.io import ascii
data = ascii.read(pathToRaveData)

yamlString = """filters: {complevel: 5, complib: blosc}
schema:
  basic:
    primary_key: oid
    spatial_keys: [RAdeg, DEdeg]
    columns:
    - [oid, u8]
"""

raveCols = ''

for idx, val in enumerate(data.colnames):
    colname = str(data.colnames[idx])
    raveCols += colname + ','
    dtype = str(data.dtype[idx])
    
    if dtype[0] == '|':
        dtype = 'a' + dtype[2:]
    elif dtype == 'int64':
        dtype = 'i8'
    elif dtype == 'float64':
        dtype = 'f8'
    yamlString += "    - [" + colname + ", " + dtype + "]\n"

raveCols = raveCols[:-1]

f = open("raveDR4.yaml", "w")
f.write(yamlString)
f.close()

In [9]:
! cat raveDR4.yaml

filters: {complevel: 5, complib: blosc}
schema:
  basic:
    primary_key: oid
    spatial_keys: [RAdeg, DEdeg]
    columns:
    - [oid, u8]
    - [RAVE_OBS_ID, a20]
    - [RAVEID, a16]
    - [RAdeg, f8]
    - [DEdeg, f8]
    - [Glon, f8]
    - [Glat, f8]
    - [HRV, f8]
    - [eHRV, f8]
    - [CorrelationCoeff, f8]
    - [PeakHeight, f8]
    - [PeakWidth, f8]
    - [CorrectionRV, f8]
    - [SkyHRV, f8]
    - [eSkyHRV, f8]
    - [SkyCorrelationCoeff, f8]
    - [ZeroPointFLAG, a2]
    - [STN_SPARV, a2]
    - [ID_TYCHO2, a2]
    - [Dist_TYCHO2, a2]
    - [XidQualityFLAG_TYCHO2, a2]
    - [pmRA_TYCHO2, a2]
    - [epmRA_TYCHO2, a2]
    - [pmDE_TYCHO2, a2]
    - [epmDE_TYCHO2, a2]
    - [ID_UCAC2, a8]
    - [Dist_UCAC2, a5]
    - [XidQualityFLAG_UCAC2, a2]
    - [pmRA_UCAC2, a5]
    - [epmRA_UCAC2, a3]
    - [pmDE_UCAC2, a4]
    - [epmDE_UCAC2, a3]
    - [ID_UCAC3, a10]
    - [Dist_UCAC3, a5]
    - [XidQualityFLAG_UCAC3, a2]
    - [pmRA_UCAC3, a5]
    - [epmRA_UCAC3, a3]
    - [pmDE_UCAC3, a

In [10]:
! lsd-admin create table --drop-existing --schema raveDR4.yaml Rave_dr4

Table 'Rave_dr4' dropped.

-------- committing 20160309185140.415509 [Rave_dr4] ---------
[Rave_dr4] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.17 sec
[Rave_dr4] Updating neighbors: Already up to date.
[Rave_dr4] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.14 sec
[Rave_dr4] Updating stats: [0 el.]>  0.00 sec
[Rave_dr4] Marking tablets read-only...
----------- success 20160309185140.415509 [Rave_dr4] ---------

Table 'Rave_dr4' created.


In [11]:
! lsd-import text -d , --cols $raveCols Rave_dr4 $pathToRaveData

Importing from 1 pieces:
/astro/apps6/opt/anaconda2.4/lib/python2.7/site-packages/lsd/bhpix.py:27: RuntimeWarning: invalid value encountered in greater
  sigz = np.where(z > 0, 2 - sqrt(3*(1-z)), -(2 - sqrt(3*(1+z))) )
/astro/apps6/opt/anaconda2.4/lib/python2.7/site-packages/lsd/bhpix.py:29: RuntimeWarning: invalid value encountered in less
  x = np.where(abs(z) < 2./3, 	phi, 		phi - (abs(sigz) - 1) * (phit - 0.25 * pi))
/astro/apps6/opt/anaconda2.4/lib/python2.7/site-packages/lsd/bhpix.py:30: RuntimeWarning: invalid value encountered in less
  y = np.where(abs(z) < 2./3, 	3./8*pi*z,	0.25 * pi * sigz)
------ rolling back 20160309185143.179332 ---------

Traceback (most recent call last):
  File "/astro/apps6/opt/anaconda2.4/bin/lsd-import", line 88, in <module>
    args.func(args)
  File "/astro/apps6/opt/anaconda2.4/bin/lsd-import", line 53, in chunk_importer
    import_from_chunks(db, importer, chunks)
  File "/astro/apps6/opt/anaconda2.4/bin/lsd-import", line 30, in import_from_chun